In [ ]:
def Jarvis() :
    #################### ▼패키지 임포트▼ ####################
    import IPython.display as ipy
    import subprocess, os
    import time, pickle
    import tensorflow as tf
    from 입력단 import input_final_code as ifc
    from 분류기 import classifier_final_code as cfc
    from 시스템.Get_Weather import get_today_weather as gtw
    from 시스템.Control_Monitor import monitor_control as mc
    from 시스템.Control_Gesture import ppt_motion_action as pma
    from 출력단 import google_tts as tts
    from 출력단.TensorFlowTTS import TTS_Tacotron2_MB_MELGAN as tts_taco
    
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding
    from tensorflow.keras.optimizers import Adam
    #################### ▲패키지 임포트▲ ####################
    
    #################### ▼파일 및 모델 로드▼ ####################
    # 단어장 로드
    with open('분류기/Save_Vocab/src_tokenizer.p', 'rb') as f:
        src_tokenizer = pickle.load(f)
    with open('분류기/Save_Vocab/tar_tokenizer.p', 'rb') as f:
        tar_tokenizer = pickle.load(f)
    
    # 모델 파라미터 정의
    max_len = 200
    embedding_dim = 128
    hidden_units = 128
    
    index_to_word = src_tokenizer.index_word
    index_to_tag = tar_tokenizer.index_word
    vocab_size = len(src_tokenizer.word_index) + 1
    tag_size = len(tar_tokenizer.word_index) + 1
    
    # 모델 로드
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, mask_zero= True))
    model.add(Bidirectional(LSTM(hidden_units, return_sequences= True)))
    model.add(TimeDistributed(Dense(tag_size, activation= ('softmax'))))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
    
    # 가중치 로드
    model.load_weights('분류기/Save_Model/bidirectional_lstm_(final)_Project_tags/bidirectional_lstm_(final)_Project_tags')
    #################### ▲파일 및 모델 로드▲ ####################
    
    #################### ▼함수 선언▼ ####################
    # 파일을 실행해주는 함수 입니다.
    def 실행및종료(path, act) :
        if act in [0, 4] :                            # act 변수의 값이 [0, 4] 안에 있다면
            os.startfile(path)                        # path 변수에 저장된 경로의 파일을 실행합니다.
        elif act in [1, 5] :                            # act 변수의 값이 [1, 5] 안에 있다면
            os.system('taskkill /f /im POWERPNT.EXE') # 프로세스명을 이용하여 프로세스를 강제 종료합니다.        
    #################### ▲함수 선언▲ ####################
    
    #################### ▼비서 메인 실행 코드▼ ####################
    # TAS에 저장된 값을 인덱스 번호로 반환하기 위한 딕셔너리 변수 입니다.
    TAS = {0: 'ppt', 1: '피피티', 2: 'powerpoint', 3: '프레젠테이션', 4: '프리젠테이션', 5: '파워포인트', 
           6: '인터넷', 7: 'internet', 8: '웹', 9: 'web', 10: '네이버', 11: 'naver', 12: '구글', 13: 'google',
           14: '컴퓨터', 15: '가상 마우스', 16: '마우스', 17: '제스처모드', 18: '읽기모드', 19: '리더모드',
           20: 'redermode', 21: '헬기모드', 22: '파일', 23: 'file', 24: '폴더', 25: 'forder', 26: '날씨', 27: '메모장'}
    
    # ACT에 저장된 값을 인덱스 번호로 반환하기 위한 딕셔너리 변수 입니다.
    ACT = {0: '켜', 1: '꺼', 2: '보여', 3: '보고', 4: '실행', 5: '종료', 6: '옮겨', 7: '이동', 8: '알려줘'}
    
    # 데스크탑 기반 인공지능 음성 비서를 수행합니다.
    while 1 :
        try :
            text = ifc.VoiceCollect() # 음성을 수집합니다. # 자비스안녕
            print('1.', text)

            # 호출어를 입력받으면 음성 비서가 기능을 수행합니다.
            if ('자비스' in text) or ('자베스' in text) or ('차베스' in text) : # 문장 속 호출어가 있는지 확인합니다.
                
                display(ipy.Audio('호출음.wav', autoplay=True))                 # 호출어가 입력되면 응답 소리를 출력합니다.
                start_all = time.time()                                         
                text = ifc.VoiceCollect()                                       # 명령을 수집합니다.
                print('2.', text)                                               # 수집된 음성을 출력하여 보여줍니다.
                start = time.time()
                
                # 분류기에서 명령문을 토큰별로 나누어 딕셔너리 형태로 반환받습니다.
                command_dict = cfc.classifier(text, src_tokenizer, tar_tokenizer, max_len, model, index_to_word, index_to_tag)
                
                # 반환 받은 딕셔너리를 출력하여 보여줍니다.
                print('3. command_dict -', command_dict)

                # ACT와 TAS의 요소가 1개 이상일 때 제대로된 명령을 받았다고 판단하고 기능을 수행합니다.
                if (len(command_dict['ACT']) > 0) and (len(command_dict['TAS']) > 0) :

                    # command_dict['TAS']의 값을 i 변수에 하나씩 넘겨주며 반복합니다.
                    for tas, act in zip(command_dict['TAS'], command_dict['ACT']) :
                        for key in TAS.keys() :  # TAS의 키 값을 key 변수에 하나씩 넘겨주며 반복합니다.
                            if TAS[key] == tas : # TAS의[key]의 값이 tas와 같다면
                                tas_idx = key    # key 변수에 저장된 키 값을 tas_idx 변수에 넘겨줍니다.
                                print(tas_idx)
                                break

                        for key in ACT.keys() :  # ACT의 키 값을 key 변수에 하나씩 넘겨주며 반복합니다.
                            if ACT[key] == act : # ACT의[key]의 값이 tas와 같다면
                                act_idx = key    # key 변수에 저장된 키 값을 act_idx 변수에 넘겨줍니다.
                                print(act_idx)
                                break

                        # 반환 반은 idx 번호를 가지고 기능을 수행합니다.
                        if (tas_idx in [0, 1, 2, 3, 4, 5]) and (act_idx in [0, 1, 2, 4, 5]) : # 파워포인트 실행/종료
                            path = 'C:/Program Files/Microsoft Office/Office15/POWERPNT.EXE'
                            실행및종료(path, act_idx)
                        elif (tas_idx in [27]) and (act_idx in [0, 1, 2, 4, 5]) :             # 메모장 실행/종료
                            path = '%windir%\system32\notepad.exe'
                            실행및종료(path, act_idx)
                        elif (tas_idx in [26]) and (act_idx in [8]) :                         # 오늘 날씨 리뷰
                            tts.GoogleTTS(gtw.TodayWeather())
                        elif (tas_idx in [18, 19, 20, 21]) and (act_idx in [0, 4]) :          # 리더 모드 실행
                            mc.MonitorControl()
                        elif (tas_idx in [17]) and (act_idx in [0, 4]) :                      # 제스쳐 모드 실행
                            pma.ppt_motion_action()
                        elif (tas_idx in [17]) and (act_idx in [1, 5]) :                      # 제스쳐 모드 종료
                            send_end.send('quit')
                else :
                    tts.GoogleTTS('잘못된 입력입니다. 다시 한번 말씀해주세요.') 
                    
            elif '종료해줘' in text : # 문장 속 '종료해줘'라는 단어가 있다면
                send_end.send('exit') # send_end 함수에 'exit' 단어를 넘겨주어 프로그램을 종료합니다.
                break                 # 반복을 종료합니다.
            else :                    # 그렇지 않다면
                continue              # 계속 반복합니다.
        except Exception as e :
            print(e)
    #################### ▲비서 메인 실행 코드▲ ####################

In [ ]:
Jarvis()

In [ ]:
!pip install tensorflow